# **ΠΡΟΒΛΗΜΑ ΛΟΓΙΣΤΙΚΗΣ ΠΑΛΙΝΔΡΟΜΗΣΗΣ – ΠΡΟΣΔΕΣΗ ΧΗΜΙΚΩΝ ΜΟΡΙΩΝ ΣΕ ΥΠΟΔΟΧΕΙΣ**

Σε αυτό το πρόβλημα υφίστανται δυαδικά και συνεχή χαρακτηριστικά. Το πρόβλημα ήδη ανάγεται ενδεικτικά στη χρήση δεντρικών ή νευρικών μοντέλων ή ακόμα και συνδυασμό τους. Στη παρούσα άσκηση χρησιμοποιείται ένα νευρωνικό δίκτυο.
Η μόνη προ-επεξεργασία που υφίστανται τα συνεχή δεδομένα είναι η κανονικοποίησή τους (για τα συνεχή χαρακτηριστικά), ενώ σε σχέση με το προηγούμενο πρόβλημα δεν έχουμε πληροφορίες που να υποδεικνύουν τη χρήση μείωσης διαστάσεων για την απλοποίηση και ταχύτερη σύγκλιση του μοντέλου.  
Μοναδικό ζήτημα που τίθεται είναι η ανισοκατανομή του πλήθος δεδομένων των 2 κλάσεων,0 και 1, με ποσοστά 33.5% και 66.5% αντιστοίχως. Αυτό συνεπάγεται πως η μετρική της ακρίβειας (accuracy) δεν αρκεί για να προσδιορίσει τη πραγματική συμπεριφορά του μοντέλου, καθώς μπορεί να υπάρχει μεγαλύτερη τάση σωστής ταξινόμησης της επικρατέστερης –αριθμητικά- κλάσης με αποτέλεσμα να δίνεται ψευδώς η αίσθηση πως ο ταξινομητής λειτουργεί αμερόληπτα και το ίδιο αποδοτικά και για τις 2 κατηγορίες.


In [6]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold


# Load Data
train_features = pd.read_csv("Train_Features.csv")
train_labels = pd.read_csv("Train_Labels.csv")
test_features = pd.read_csv("Test_Features.csv")

scaler = StandardScaler()

# See the percentage of class 1 to determine dataset imbalances
print(f"Samples that binded to the receptor are {np.sum(train_labels == 1) / len(train_labels) * 100}%")

def preprocess_data(features):
    # Separate continuous and binary features
    continuous = features.iloc[:, :1425]
    binary = features.iloc[:, 1425:]
    # Scale only continuous features (duh)
    continuous_scaled = scaler.fit_transform(continuous)

    return np.hstack((continuous_scaled, binary.values))

# Preprocess training data
X = preprocess_data(train_features)
y = train_labels.values.ravel()  # Flatten the 2D vector

# Split data for validation / stratify to keep same distribution in train and validation / not used eventually
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)




Samples that binded to the receptor are 1    66.427289
dtype: float64%


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Η δομή καθορίστηκε με γενικές κατευθυντήριες και πειραματισμό.
   Προτιμήθηκαν περισσότερα layers με λιγότερους νευρώνες σε καθένα.

- Κάθε layer είναι Dense με ReLU για ενεργοποίηση

Batch normalization για ταχύτερη σύγκλιση,περιορισμό  
  μεγάλων εξόδων, αποφυγή overfitting & dropout neurons,
  για καλύτερη απόδοση στο validation & testing phase πριν το τέλος
  δικτύου πάλι για αποφυγή overfitting

- Σιγμοειδής στην έξοδο

Φτιάχνεται ένα πλέγμα από παραμέτρους στη συνέχεια το οποίο θα χρησιμοποιηθεί για να βρεθεί το βέλτιστο set που δίνει το καλύτερο μοντέλο βάση μετρικής/μετρικών που υπολογίζονται.


In [7]:


# Define the Neural Network
class ReceptorBindPredictor(nn.Module):
    def __init__(self, dr):
        super(ReceptorBindPredictor, self).__init__()
        self.model = nn.Sequential(
          nn.Linear(X_train.shape[1], 1),
          #nn.ReLU(),
          #nn.Linear(64, 64),
          #nn.ReLU(),
          #nn.Linear(64, 64),
          #nn.ReLU(),
          #nn.Linear(64, 32),
          #nn.ReLU(),
          #nn.Dropout(dr),
          #nn.BatchNorm1d(32),
          nn.Linear(1, 1),
          nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x


# Calculate combined score
def calculate_combined_score(auc, accuracy, loss, auprc):
    combined_score = (0.26 * auc +
                      0.23 * accuracy -
                      0.24 * loss +
                      0.27 * auprc)

    return combined_score

# Define the grid of hyperparameters
param_grid = {
    'lr': [0.001, 0.005],
    'dropout_rate': [0.2, 0.3, 0.4],
    'num_epochs': [50],
    'batch_size': [32, 64]
}


Για την εξασφάλιση σταθερής απόδοσης γίνεται και πάλι k cross validation, έτσι ώστε το σύστημα να είναι αξιόπιστο και η εκπαίδευσή του να μην εξαρτάται από τον τυχαίο διαμοιρασμό των batches, αλλά και των διαχωρισμό των δεδομένων σε training και validation set.
Οι μετρικές που υπολογίζονται είναι, το σφάλμα με Binary Cross
   Entropy, accuracy, AUC και AUPRC.
Εδώ κάθε μετρική φέρει διαφορετική βαρύτητα,
   λόγω των δεδομένων του προβλήματος.
Γίνεται μια συνάρτηση που συμψηφίζει τις μετρικές. Τα βάρη
   ρυθμίζονται εμπειρικά, αλλά και σύμφωνα με τις σχετικές και
   απόλυτες αποκλίσεις κάθε μετρικής.
   
   Αυτή η τεχνική δεν βρέθηκε σε κάποια βιβλιογραφική πηγή, αλλά ήταν μια τελείως πειραματική μέθοδος που θυμίζει όμοιες τεχνικές που χρησιμοποιούνται στη μηχανική εντός και εκτός του πεδίου του AI (όπως είναι το weighted voting στα ensemble μοντέλα, αλλά και σε εφαρμογές φίλτρων για noise reduction αντίστοιχα)



In [8]:

best_combined_score = -1  # Initialize with lowest value
best_model = None
best_params = None
best_val_loss = None
best_val_auc = 0
best_val_acc = 0
best_val_auprc = 0
criterion = nn.BCELoss()

kf = StratifiedKFold(n_splits=5, shuffle=True)  # 5-fold cross-validation

for parameters in ParameterGrid(param_grid):
    fold_metrics = []  # Track metrics for each fold

    for train_idx, val_idx in kf.split(X, y):
        # Split data
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Create DataLoaders
        train_dataset = TensorDataset(torch.tensor(X_train_fold, dtype=torch.float32),
                                      torch.tensor(y_train_fold, dtype=torch.float32))
        val_dataset = TensorDataset(torch.tensor(X_val_fold, dtype=torch.float32),
                                    torch.tensor(y_val_fold, dtype=torch.float32))
        train_loader = DataLoader(train_dataset, batch_size=parameters['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=parameters['batch_size'])

        # Initialize and train model
        model = ReceptorBindPredictor(parameters['dropout_rate'])
        optimizer = optim.Adam(model.parameters(), lr=parameters['lr'])

        for epoch in range(parameters['num_epochs']):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad() # To prevent gradient contamination
                predictions = model(X_batch).squeeze() # Feed data and get predictions ε[0,1]
                loss = criterion(predictions, y_batch) # to assess error
                loss.backward() # Update weights based on gradients
                optimizer.step() # Update model parameters

        # Validation metrics
        model.eval() # Switch to evaluation mode - disable dropout for consistent results / enable batchNorm for running statistics
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                y_pred = model(X_batch).squeeze()
                val_predictions.extend(y_pred.numpy())
                val_targets.extend(y_batch.numpy())

        val_auc = roc_auc_score(val_targets, val_predictions)
        val_accuracy = accuracy_score(val_targets, (np.array(val_predictions) > 0.5).astype(int))
        val_loss = criterion(torch.tensor(val_predictions), torch.tensor(val_targets)).item()
        val_auprc = average_precision_score(val_targets, val_predictions)

        # Store metrics
        fold_metrics.append({
            'auc': val_auc,
            'accuracy': val_accuracy,
            'loss': val_loss,
            'auprc': val_auprc
        })

    # Average metrics across folds
    avg_auc = np.mean([m['auc'] for m in fold_metrics])
    avg_accuracy = np.mean([m['accuracy'] for m in fold_metrics])
    avg_loss = np.mean([m['loss'] for m in fold_metrics])
    avg_auprc = np.mean([m['auprc'] for m in fold_metrics])
    combined_score = calculate_combined_score(avg_auc, avg_accuracy, avg_loss, avg_auprc)

    # Update best model if necessary
    if combined_score > best_combined_score:
        best_combined_score = combined_score
        best_model = model
        best_params = parameters
        best_val_auc = avg_auc
        best_val_acc = avg_accuracy
        best_val_loss = avg_loss
        best_val_auprc = avg_auprc

# Print the best hyperparameters found
print(f"Best Hyperparameters: {best_params}")
print(f"Best model has Val Loss: {best_val_loss/len(val_loader):.4f}, Val AUC: {best_val_auc:.4f}, Val AUPRC: {val_auprc:.4f}, Val Accuracy: {best_val_acc:.4f}")



Best Hyperparameters: {'batch_size': 64, 'dropout_rate': 0.3, 'lr': 0.001, 'num_epochs': 50}
Best model has Val Loss: 0.0643, Val AUC: 0.9557, Val AUPRC: 0.9685, Val Accuracy: 0.9093


Τέλος παράγονται τα εκτιμώμωνα labels του προβλήματος και υλοποιείται μια συνάρτηση που για κάθε επόμενο test signal που μπορεί να δωθεί, να γίνεται απευθείας η εκτίμηση της κλάσης του.

In [9]:
# Preprocess test data
test_features_aligned = test_features.reindex(columns=train_features.columns, fill_value=0)
X_test = preprocess_data(test_features_aligned)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Generate submission file
def generate_submission_file(model, X_test_tensor, file_name):
    model.eval()
    with torch.no_grad():
        test_probabilities = model(X_test_tensor).squeeze().numpy() # Make it 2D and numpy
    test_predictions = (test_probabilities > 0.5).astype(int)
    submission = pd.DataFrame({
        'predicted_label': test_predictions,
        'prediction_score': test_probabilities
    })
    submission.to_csv(file_name, index=False)

output_file = "test_predictions_task2_58352.csv"
generate_submission_file(best_model, X_test_tensor, output_file)

**ΣΥΜΠΕΡΑΣΜΑΤΑ ΚΑΙ ΠΑΡΑΔΟΧΕΣ**

Εντοπίστηκαν αρκετά προβλήματα εκ του αποτελέσματος, που δεν επέτρεψαν το μοντέλο να εκπαιδευτεί σωστά και οδήγησαν τις μετρικές να επιστρέφουν ψευδώς οπτιμιστικά αποτελέσματα.

1) Μηδαμινό αλλά υπαρκτό σφάλμα ήταν το ότι ο τρόπος που διαβάζονταν τα δεδομένα έπιαναν τη πρώτη σειρά από κάθε CSV ως επικεφαλίδα και όχι ως δείγμα. Αυτό διορθωνόταν με ένα flag εντός της συνάρτησης (header=None)

2) Υπήρχε κατά κάποιο τρόπο data leakage, καθώς γινόταν normalisation σε όλο το training set, αλλά όχι σε κάθε fold μεμονωμένα, με αποτέλεσμα να δίνονται πληροφορίες για τη μέση τιμή και τυπική απόκλιση των χαρακτηριστικών από το - διαφορετικό κάθε φορά- validation set, οπότε και τα αποτελέσματα κατά το validation phase ήταν ψευδή.

3) Δεν έγινε χρήση του stratify στην KFOLD, (π.χ χρήση της StratifiedKFold), οπότε το class imbalance δεν ήταν κάθε στιγμή της εκπαίδευσης κοινό.

4) Η αποφυγή χρήσης PCA (αν και δικαιολογημένη σε πρώτη όψη, διότι δεν είχαμε κάποια σημασιολογική πληροφορία που να υποδείξει πως η μείωση διαστάσεων θα ήταν θεμιτή), μπορεί να είχε ως αποτέλεσμα να εκπαιδεύεται το μοντέλο με δεδομένα τα οποία να παρουσιάζουν αμελητέο variance, και οπότε να σηματοδοτούν πως ίσως να απεικονίζουν θόρυβο που να οδηγούν ενδεχομένως σε overfitting

5) Η χρήση dropout ακριβώς πριν την έξοδο, ψαλίδιζε σημαντική πληροφορία σε κρίσιμο σημείο του δικτύου, οπότε η αποφάσεις του μοντέλου μπορεί να παρουσίαζαν τυχαιότητα.

6) Η εμπειρική μεθοδολογία αξιολόγησης μοντέλου, όντας τελείως πειραματική, άρα και κάθολου βέλτιστα παραμετροποιημένη, μπορεί να οδήγησε στην επιλογή πολύ χειρότερου μοντέλου από τα διαθέσιμα που εκπαιδεύτηκαν.

Ωστόσο έγινε προσπάθεια, σε συνέχεια της παρουσίασης της τελικής εργασίας, να λυθούν αυτά τα προβλήματα σε ξεχωριστό notebook, αλλά κάποια από αυτά συνέχισαν να εμφανίζονται